# Multisession registration with CaImAn

This notebook will help to demonstrate how to use CaImAn on movies recorded in multiple sessions. CaImAn has in-built functions that align movies from two or more sessions and try to recognize components that are imaged in some or all of these recordings.

The basic function for this is `caiman.base.rois.register_ROIs()`. It takes two sets of spatial components and finds components present in both using an intersection over union metric and the Hungarian algorithm for optimal matching.
`caiman.base.rois.register_multisession()` takes a list of spatial components, aligns sessions 1 and 2, keeps the union of the matched and unmatched components to register it with session 3 and so on.

In [44]:
from IPython import get_ipython
from matplotlib import pyplot as plt
import numpy as np
import pickle
import h5py
import glob
import pims_nd2
import pandas as pd
from collections import OrderedDict
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns

from caiman.base.rois import register_multisession
from caiman.utils import visualization
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.utils.utils import download_demo

import sys
sys.path.append('..')
from placecode.cross_registration_functions import CellTrackingMultipleAnimals as celltrack


try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


loading the files and trying to make everything automatic

In [34]:
animals=['OPI2342','OPI2356']
conditions=['baseline_d1','baseline_d2','24hrs_post_SD']

In [35]:
files_list=OrderedDict()
templates_list=OrderedDict()
cnmf_list=OrderedDict()
spatials=OrderedDict()
dims=OrderedDict()
assignments=OrderedDict()
cr_reg_cells=OrderedDict()
cr_reg_cells_df=OrderedDict()

for animal in animals:      

    # Load multisession data (spatial components and mean intensity templates) (should be replaced by actual data)
    home_folder=f'D:/sd_project_pbox/exported_data/{animal}/'
    raw_data_folder=f'D:/sd_project_pbox/raw_data/{animal}'
    output_folder=f'D:/sd_project_pbox/results/{animal}/'

    files_list[animal]=[]
    templates_list[animal]=[]
    cnmf_list[animal]=[]
    spatials[animal]=[]
    dims[animal]=[]
    assignments[animal]=[]
    cr_reg_cells[animal]=[]
    cr_reg_cells_df[animal]=[]
# spatials=[cnmf_1.estimates.A,cnmf_2.estimates.A,cnmf_3.estimates.A]
# dims=cnmf_1.dims

    for condition in conditions:

        #finding file paths
        fpath= glob.glob(f"{home_folder}/{condition}/*.hdf5")[0]
        files_list[animal].append(fpath)

        #sstoring cnfs
        cnmf_ind=cnmf.load_CNMF(fpath,'r')
        cnmf_list[animal].append(cnmf_ind)
        #sotring spatial components
        spatial=cnmf_ind.estimates.A
        spatials[animal].append(spatial)
        #storing dims
        dim=cnmf_ind.dims
        dims[animal].append(dim)

        #template paths
        template_path=glob.glob(f"{raw_data_folder}/*{condition}/*.nd2")[0]
        nikon_movie=pims_nd2.ND2_Reader(template_path)
        template=np.mean(nikon_movie[:600],axis=0)
        templates_list[animal].append(template)


c:\Users\ttami\anaconda3\envs\caiman\lib\site-packages\pims\base_frames.py:478: UserWarning:

Please call FramesSequenceND.__init__() at the start of thethe reader initialization.



## Use `register_multisession()`

The function `register_multisession()` requires 3 arguments:
- `A`: A list of ndarrays or scipy.sparse.csc matrices with (# pixels X # component ROIs) for each session
- `dims`: Dimensions of the FOV, needed to restore spatial components to a 2D image
- `templates`: List of ndarray matrices of size `dims`, template image of each session

In [36]:
for animal in animals:
    spatial_union, assignments_per_animal, matchings = register_multisession(A=spatials[animal], dims=dims[animal][0])
    assignments[animal]=assignments_per_animal


c:\Users\ttami\anaconda3\envs\caiman\lib\site-packages\scipy\sparse\_index.py:151: SparseEfficiencyWarning:

Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.



FInd the rows where there are no NaN values. Meaning find cells that are cross registered across all conditions

In [37]:
# Find the rows where there are no NaN values
#meaning where are the cells in all of the days

for animal in animals:    
    cr_reg_cells[animal] = assignments[animal][~np.isnan(assignments[animal]).any(axis=1)].astype(int)
    cr_reg_cells_df[animal]=pd.DataFrame(data=cr_reg_cells[animal],columns=conditions)


In [38]:
#first store the place cells somewhere
pc_cells=OrderedDict()
non_pc_cells=OrderedDict()
silent_cells=OrderedDict()

for animal in animals:
    output_file=f"D:/sd_project_pbox/results/{animal}"


    for condition in conditions:
        pc_cells[f'{animal}_{condition}']=[]
        non_pc_cells[f'{animal}_{condition}']=[]
        silent_cells[f'{animal}_{condition}']=[]

        #appending place cells
        pc_cells_per_day=h5py.File(f'{output_file}/{condition}/output_info.h5')['place_cells_tuned_vector']
        pc_cells_per_day=np.array(pc_cells_per_day)
        pc_cells[f'{animal}_{condition}'].append(pc_cells_per_day)

        #appending non place cells
        non_pc_cells_per_day=h5py.File(f'{output_file}/{condition}/output_info.h5')['non_place_cells_tuned_vector']
        non_pc_cells_per_day=np.array(non_pc_cells_per_day)
        non_pc_cells[f'{animal}_{condition}'].append(non_pc_cells_per_day)

        #appending silent cells
        silent_cells_per_day=h5py.File(f'{output_file}/{condition}/output_info.h5')['silent_cells']
        silent_cells_per_day=np.array(silent_cells_per_day)
        silent_cells[f'{animal}_{condition}'].append(silent_cells_per_day)


In [45]:
labels = [] 
colors = []
for condition in conditions:
  labels.extend([f"PC {condition}", f"nPC {condition}", f"sil {condition}"])  # for each condition, check categories PC, not-PC and low activity
  colors.extend(["red", "blue", "grey"])  # 255, 0, 0;  0, 255, 0; 0, 0, 0
# in each condition, we have PC and nPC categories, each have PC and nPC targets in the next category
sources = []  # should be 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 4, 5, 3, 4, 5, 3, 4, 5, ...
targets = []  # should be 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, ...
values = []
link_colors = []
#initlaiizng zero for all the values so that I can add numbers for individual animals
n_PC_to_PC=n_non_PC_to_PC=n_sil_to_PC=0
n_PC_to_non_PC=n_sil_to_non_PC=n_non_PC_to_non_PC=0
n_PC_to_sil=n_non_PC_to_sil=n_sil_to_sil=0

for animal in animals:
  df=cr_reg_cells_df[animal]
  for i_condition in range(len(conditions)-1):

    #target cells place cells
    n_PC_to_PC+=len(celltrack.cell_movement(df,animal,conditions[i_condition],conditions[i_condition+1],source_type=pc_cells,target_type=pc_cells))
    n_non_PC_to_PC+=len(celltrack.cell_movement(df,animal,conditions[i_condition],conditions[i_condition+1],source_type=non_pc_cells,target_type=pc_cells))
    n_sil_to_PC+=len(celltrack.cell_movement(df,animal,conditions[i_condition],conditions[i_condition+1],source_type=silent_cells,target_type=pc_cells))
    if animal==animals[-1]:
      sources.extend([3*i_condition,3*i_condition+1, 3*i_condition+2])
      targets.extend([3*(i_condition+1), 3*(i_condition+1), 3*(i_condition+1)])
      values.extend([n_PC_to_PC, n_non_PC_to_PC, n_sil_to_PC])
      link_colors.extend(["rgba(255, 0, 0, 0.4)", "rgba(0, 0, 255, 0.4)", "rgba(0, 0, 0, 0.4)"])  # PC -> x is light blue, nPC -> x is light red, Q -> y is "light black"

    #target cells non place cells
    n_PC_to_non_PC+=len(celltrack.cell_movement(df,animal,conditions[i_condition],conditions[i_condition+1],source_type=pc_cells,target_type=non_pc_cells))
    n_sil_to_non_PC+=len(celltrack.cell_movement(df,animal,conditions[i_condition],conditions[i_condition+1],source_type=silent_cells,target_type=non_pc_cells))
    n_non_PC_to_non_PC+=len(celltrack.cell_movement(df,animal,conditions[i_condition],conditions[i_condition+1],source_type=non_pc_cells,target_type=non_pc_cells))
    if animal==animals[-1]:
      sources.extend([3*i_condition,3*i_condition+1, 3*i_condition+2])
      targets.extend([3*(i_condition+1)+1, 3*(i_condition+1)+1, 3*(i_condition+1)+1])
      values.extend([n_PC_to_non_PC, n_non_PC_to_non_PC, n_sil_to_non_PC])
      link_colors.extend(["rgba(255, 0, 0, 0.4)", "rgba(0, 0, 255, 0.4)", "rgba(0, 0, 0, 0.4)"])  # PC -> x is light blue, nPC -> x is light red, Q -> y is "light black"


    #targer cell silent cells  
    n_PC_to_sil+=len(celltrack.cell_movement(df,animal,conditions[i_condition],conditions[i_condition+1],source_type=pc_cells,target_type=silent_cells))
    n_non_PC_to_sil+=len(celltrack.cell_movement(df,animal,conditions[i_condition],conditions[i_condition+1],source_type=non_pc_cells,target_type=silent_cells))
    n_sil_to_sil+=len(celltrack.cell_movement(df,animal,conditions[i_condition],conditions[i_condition+1],source_type=silent_cells,target_type=silent_cells))
    if animal==animals[-1]:
      sources.extend([3*i_condition,3*i_condition+1, 3*i_condition+2])
      targets.extend([3*(i_condition+1)+2, 3*(i_condition+1)+2, 3*(i_condition+1)+2])
      values.extend([n_PC_to_sil, n_non_PC_to_sil, n_sil_to_sil])
      link_colors.extend(["rgba(255, 0, 0, 0.4)", "rgba(0, 0, 255, 0.4)", "rgba(0, 0, 0, 0.4)"])  # PC -> x is light blue, nPC -> x is light red, Q -> y is "light black"

  
fig = go.Figure(data=[go.Sankey(
  node = dict(
    pad = 15,
    thickness = 20,
    line = dict(color = "black", width = 0.5),
    label = labels,
    color = colors
  ),
  link = dict(
    source = sources, # indices correspond to labels, eg A1, A2, A1, B1, ...
    target = targets,
    value = values,
    color = link_colors
))])

fig.update_layout(title_text=f'Multianimal session tracking\n{animals}\n\n{conditions}', font_size=10)

#pio.write_image(fig, F'{output_folder}/{animal}_sankey_diagram.pdf')



In [42]:
len(cr_reg_cells['OPI2356'])

59

In [48]:
cr_reg_cells_df['OPI2342']['baseline_d2']

0     23
1     30
2     41
3     93
4    100
5     34
6     97
7    104
Name: baseline_d2, dtype: int32

In [29]:
cr_reg_cells_df

OrderedDict([('OPI2342',
                 baseline_d1  baseline_d2  24hrs_post_SD
              0           26           23             89
              1           32           30             16
              2           42           41             26
              3           54           93             35
              4          103          100             91
              5          120           34             18
              6          145           97             14
              7          147          104             87),
             ('OPI2356',
                  baseline_d1  baseline_d2  24hrs_post_SD
              0            22           20             27
              1            24           22             28
              2            25          209            215
              3            29           28             37
              4            31           30             39
              5            33           38             44
              6            41

In [30]:
#define cross registered place cells per session

def filter_cross_registered_place_cells(df, sessions, place_cell_arrays):
    if not sessions:
        return df  # Return the original DataFrame if no sessions are specified
    
    mask = df[sessions[0]].isin(place_cell_arrays[sessions[0]][0])
    for session in sessions[1:]:
        mask &= df[session].isin(place_cell_arrays[session][0])
    
    filtered_df = df[mask]
    return filtered_df[sessions]

def cell_movement(df,source_condition,targ_condition,source_type,target_type):
    
    mask = df[source_condition].isin(source_type[source_condition][0])
    mask &= df[targ_condition].isin(target_type[targ_condition][0])

    filtered_df = df[mask]
    return filtered_df[[source_condition,targ_condition]]

In [31]:

sessions=conditions #all of them
sessions=['baseline_d1','baseline_d2']
sessions=['baseline_d2','24hrs_post_SD']
#sessions=conditions

# Example usage for two sessions:
filtered_df = filter_cross_registered_place_cells(df, sessions, pc_cells)


comparidson between ['baseline_d2', '24hrs_post_SD']
Empty DataFrame
Columns: [baseline_d2, 24hrs_post_SD]
Index: []


In [32]:
#indetify only the cross registered place cells in each of the recordings
fr_rate_maps_to_compare=OrderedDict()

for session in sessions:
    fr_rate_maps_to_compare[session]=[]

for session in sessions:
    avr_fr_rate_map=h5py.File(f'{output_folder}/{session}/output_info.h5')['avr_firing_rate_maps']
    avr_fr_rate_map=pd.DataFrame(avr_fr_rate_map)
    cr_reg_cells_values=filtered_df[session].values
    avr_fr_rate_map=avr_fr_rate_map.iloc[cr_reg_cells_values]
    fr_rate_maps_to_compare[session]=avr_fr_rate_map



In [ ]:
#plotting the original place cell coding diagrams

sel_fontize=20

fig,ax=plt.subplots(2,len(sessions),figsize=(20,20))
fig.suptitle(f'{animal}\ncross registered place cells\n comparison{sessions}\n Up Original sorting\nBelow sorted to {sessions[0]}',fontsize=sel_fontize+5)
                    
custom_ticks = [0, 50, 100, 150]  # Specify the positions where you want the ticks
custom_labels = ['0', '50', '100', '150']  

for i,session in enumerate(sessions):
    place_cell_activity=fr_rate_maps_to_compare[session]
    place_cell_indexes_max=np.argmax(place_cell_activity,axis=1) #finding the place of the max activity of every cell
    place_cell_indexed_filtered=place_cell_activity.apply(np.argmax, axis=1).sort_values().index #there are the new indexes of the place cells just so thez can form the nice place cell diagram
    transpose=place_cell_indexed_filtered
    place_cell_activity=place_cell_activity.reindex(index=transpose) #reindexing the place cell panda frame according to the max activitz and the position in the belt


    sns.heatmap(place_cell_activity,ax=ax[0,i],cmap='viridis',cbar=False)#,vmax=4)
    if i==0:
        ax[0,i].set_ylabel('cross registered place cells',fontsize=sel_fontize)
    else:
        ax[0,i].set_ylabel('')
    ax[0,i].set_xlabel('')
    ax[0,i].set_yticks([])
    ax[0,i].set_yticklabels([])
    ax[0,i].set_title(session,fontsize=sel_fontize)
    ax[0,i].set_xticks([])
    ax[0,i].set_xticklabels([])

#plotting place coding diagram transposed on the first one

place_cell_activity=fr_rate_maps_to_compare[sessions[0]].reset_index(drop=True)
place_cell_indexes_max=np.argmax(place_cell_activity,axis=1) #finding the place of the max activity of every cell
place_cell_indexed_filtered=place_cell_activity.apply(np.argmax, axis=1).sort_values().index 
transpose=place_cell_indexed_filtered

for i,session in enumerate(sessions):
    place_cell_activity=fr_rate_maps_to_compare[session].reset_index(drop=True)
    place_cell_activity=place_cell_activity.reindex(index=transpose) #reindexing the place cell panda frame according to the max activitz and the position in the belt

    sns.heatmap(place_cell_activity,ax=ax[1,i],cmap='viridis',cbar=False)
    if i==0:
        ax[1,i].set_ylabel('cross registered place cells',fontsize=sel_fontize)
    else:
        ax[1,i].set_ylabel('')
    ax[1,i].set_xlabel('Belt(cm)',fontsize=sel_fontize)
    ax[1,i].set_yticks([])
    ax[1,i].set_yticklabels([])
    ax[1,i].set_xticks(custom_ticks)
    ax[1,i].set_xticklabels(custom_labels,fontsize=sel_fontize)

plt.tight_layout()

plt.savefig(f'{output_folder}/cross_registration/{animal}_{sessions}_place_cells_comparison.png', format='png', dpi=300, bbox_inches='tight')
plt.savefig(f'{output_folder}/cross_registration/{animal}_{sessions}_place_cells_comparison.pdf', format='pdf', dpi=300, bbox_inches='tight')